# Data Collection and Cleaning - part 1
### Advanced Statistics
For each NBA season beginning with 2000 and ending with 2023, every team's advanced statistics is scraped from [basketball-reference](https://www.basketball-reference.com) and collected into a Pandas dataframe. This dataframe is passed through a function that cleans the data and prepares it for analysis. 

The advanced statistics do not include basic 3-point shooting data, but it will be used in the analysis. Therefore, the 3-point data will have to be collected separately using the same process and merged with the advanced statistics dataset. For the sake of organization and to work around the *Too Many Requests* error when scraping the data, this process will be performed in a separate notebook.

In [1]:
import pandas as pd

In [2]:
def clean_df(df):
    ''' 
    Input: raw Pandas DataFrame with each team's advanced statistics of a given year
    Output: cleaned Pandas DataFrame

    Tasks:
    - Removes asterisk from playoff team names and represent this as a new column
    - Calculate each team's winning percentage and add as new column
    - remove league average row and keep statistics only from specific teams
    '''
    df.columns = df.columns.droplevel()
    season_results = []
    for i in df['Team']:
        if '*' in i:
            season_results.append('Playoffs')
        else:
            season_results.append('Did not qualify')
    df['Season Result'] = season_results
    df['Team'] = df['Team'].apply(lambda x: x.replace('*', '')) # removes asterisk from team names
    df.dropna(axis=1, how='all', inplace=True) # remove blank columns
    win_pct = df['W']/(df['W']+df['L'])
    win_pct = win_pct.round(decimals=3)
    df.insert(5, 'Win%', win_pct) # insert new column of calculated win percentage
    df = df[:-1] # removes league average row
    
    return df

In [3]:
years=range(2000,2024)
with pd.ExcelWriter('project_data.xlsx') as writer: 
    for year in years:
        # handles issue where the indexes of the advanced stats table differ from 2013-15 and 2016-23
        if year > 2015:
            index = 10
        else:
            index = 8
        # collect scraped data
        df = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{year}.html#advanced-team')[index]
        df = clean_df(df)
        df['Year'] = year # insert new column that labels the respective year for each row of the dataframe
        
        # labels each year's champ and runner up
        champs_df = pd.read_excel('champions.xlsx', index_col=0) # excel file w/ list of NBA champions + runners-up         
        champion = champs_df.loc[year]['Champion']
        runner_up = champs_df.loc[year]['Runner-Up']
        df['Season Result'][(df['Team'] == champion) & (df['Year'] == year)] = 'Champion'
        df['Season Result'][(df['Team'] == runner_up) & (df['Year'] == year)] = 'Runner-Up'
        
        # writes the dataframes of each year to their own sheet in the excel file
        df.to_excel(writer, sheet_name=f'{year} Adv Stats', index=False)

/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season Result'][(df['Team'] == champion) & (df['Year'] == year)] = 'Champion'
/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season Result'][(df['Team'] == runner_up) & (df['Year'] == year)] = 'Runner-Up'
/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu

/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season Result'][(df['Team'] == champion) & (df['Year'] == year)] = 'Champion'
/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season Result'][(df['Team'] == runner_up) & (df['Year'] == year)] = 'Runner-Up'
/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docu

/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season Result'][(df['Team'] == champion) & (df['Year'] == year)] = 'Champion'
/var/folders/__/v5pb4rj10p5_6crzwysgscqm0000gn/T/ipykernel_36080/3113425922.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Season Result'][(df['Team'] == runner_up) & (df['Year'] == year)] = 'Runner-Up'


In [4]:
df

,Rk,Team,Age,W,L,Win%,PW,PL,MOV,SOS,...,FT/FGA,eFG%,TOV%,DRB%,FT/FGA,Arena,Attend.,Attend./G,Season Result,Year
0,1.0,Boston Celtics,27.4,57.0,25.0,0.695,57,25,6.52,-0.15,...,0.197,0.528,11.3,78.5,0.180,TD Garden,766240,18689,Playoffs,2023
1,2.0,Cleveland Cavaliers,25.4,51.0,31.0,0.622,55,27,5.38,-0.15,...,0.206,0.535,14.4,76.3,0.210,Rocket Mortgage Fieldhouse,777280,18958,Playoffs,2023
2,3.0,Philadelphia 76ers,28.2,54.0,28.0,0.659,52,30,4.32,0.06,...,0.250,0.541,13.0,77.2,0.217,Wells Fargo Center,839261,20470,Playoffs,2023
3,4.0,Memphis Grizzlies,24.4,51.0,31.0,0.622,51,31,3.94,-0.34,...,0.190,0.526,13.1,75.9,0.206,FedEx Forum,707836,17264,Playoffs,2023
4,5.0,Milwaukee Bucks,29.8,58.0,24.0,0.707,50,32,3.63,-0.02,...,0.184,0.520,10.4,77.8,0.175,Fiserv Forum,718786,17531,Playoffs,2023
5,6.0,Denver Nuggets,26.6,53.0,29.0,0.646,49,33,3.33,-0.29,...,0.194,0.543,12.2,76.4,0.201,Ball Arena,788635,19235,Champion,2023
6,7.0,New York Knicks,24.5,47.0,35.0,0.573,48,34,2.93,0.06,...,0.217,0.536,11.4,77.1,0.210,Madison Square Garden (IV),795110,19393,Playoffs,2023
7,8.0,Sacramento Kings,25.4,48.0,34.0,0.585,47,35,2.65,-0.35,...,0.225,0.563,12.6,77.2,0.203,Golden 1 Center,715491,17451,Playoffs,2023
8,9.0,Phoenix Suns,28.1,45.0,37.0,0.549,46,36,2.07,0.01,...,0.191,0.532,12.9,76.0,0.234,Footprint Center,699911,17071,Playoffs,2023
9,10.0,New Orleans Pelicans,25.9,42.0,40.0,0.512,46,36,1.89,-0.26,...,0.221,0.543,13.4,77.4,0.212,Smoothie King Center,687691,16773,Playoffs,2023


## Creating a single dataset
At this point, each year's data was exported to their own separate sheet in the same Excel file. In order to achieve a single dataset to perform the analysis, each year's data is concatenated into a single dataframe, which is exported to a separate Excel file, named ***"merged.xlsx"***.

In [5]:
f = 'project_data.xlsx'
df = pd.read_excel(f, sheet_name=None, index_col=None)

In [6]:
# concatenate all of the data from each sheet of the Excel file into a single dataframe
cdf = pd.concat(df.values())
cdf

,Rk,Team,Age,W,L,Win%,PW,PL,MOV,SOS,...,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G,Season Result,Year
0,1,Los Angeles Lakers,29.2,67,15,0.817,64,18,8.55,-0.14,...,0.241,0.443,13.4,73.1,0.222,STAPLES Center,771420.0,18815.0,Champion,2000
1,2,Portland Trail Blazers,29.6,59,23,0.720,59,23,6.40,-0.04,...,0.240,0.461,13.8,72.4,0.217,Rose Garden Arena,835078.0,20368.0,Playoffs,2000
2,3,San Antonio Spurs,30.9,53,29,0.646,58,24,5.94,-0.02,...,0.258,0.451,13.5,73.0,0.188,Alamodome,884450.0,21694.0,Playoffs,2000
3,4,Phoenix Suns,28.6,53,29,0.646,56,26,5.22,0.02,...,0.217,0.454,15.7,70.5,0.245,America West Arena,773115.0,18856.0,Playoffs,2000
4,5,Utah Jazz,31.5,55,27,0.671,54,28,4.46,0.05,...,0.260,0.477,15.0,73.2,0.256,Delta Center,801268.0,19543.0,Playoffs,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26,Portland Trail Blazers,25.1,33,49,0.402,31,51,-4.01,0.05,...,0.230,0.563,12.1,74.9,0.217,Moda Center,767374.0,18716.0,Did not qualify,2023
26,27,Charlotte Hornets,25.3,27,55,0.329,26,56,-6.24,0.35,...,0.195,0.544,12.5,75.5,0.211,Spectrum Center,702052.0,17123.0,Did not qualify,2023
27,28,Houston Rockets,22.1,22,60,0.268,23,59,-7.85,0.24,...,0.215,0.564,11.8,75.8,0.218,Toyota Center,668865.0,16314.0,Did not qualify,2023
28,29,Detroit Pistons,24.1,17,65,0.207,22,60,-8.22,0.49,...,0.227,0.557,11.9,74.0,0.231,Little Caesars Arena,759715.0,18596.0,Did not qualify,2023


In [8]:
# export concatenated dataframe to a new, separate Excel file
cdf.to_excel('merged.xlsx', sheet_name="Data",index=False)

## What's next?
This dataset is ready for the next step, where the process is repeated to collect each team's 3pt data and merged again to create the final dataset.